In [1]:
import time

import numpy as np
import pandas as pd
import torch
from torch import nn
from tqdm import tqdm

# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [2]:
# turn off warning
import logging
logging.getLogger('pyomo.core').setLevel(logging.ERROR)

## Problem Setting

In [3]:
# init
num_var = 20      # number of variables
num_ineq = 20     # number of constraints
num_data = 10000  # number of data
test_size = 1000  # number of test size
val_size = 1000   # number of validation size
train_size = num_data - test_size - val_size

In [4]:
# data sample from uniform distribution
b_samples = torch.from_numpy(np.random.uniform(-1, 1, size=(num_data, num_ineq))).float()
data = {"b":b_samples}
# data split
from src.utlis import data_split
data_train, data_test, data_dev = data_split(data, test_size=test_size, val_size=val_size)

In [5]:
# torch dataloaders
from torch.utils.data import DataLoader
batch_size = 64
loader_train = DataLoader(data_train, batch_size, num_workers=0, collate_fn=data_train.collate_fn, shuffle=True)
loader_test = DataLoader(data_test, batch_size, num_workers=0, collate_fn=data_test.collate_fn, shuffle=False)
loader_dev = DataLoader(data_dev, batch_size, num_workers=0, collate_fn=data_dev.collate_fn, shuffle=False)

## Exact Solver

In [6]:
from src.problem import msNonconvex
model = msNonconvex(num_var, num_ineq, timelimit=60)

## Heuristic - Round

In [7]:
from src.heuristic import naive_round

In [ ]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model.set_param_val({"b":b.cpu().numpy()})
    # relax
    model_rel = model.relax()
    # solve
    tick = time.time()
    xval_rel, _ = model_rel.solve("scip")
    xval, objval = naive_round(xval_rel, model)
    tock = time.time()
    # eval
    params.append(list(b))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_heur_rnd_20-20.csv")

 52%|█████████████████████████████████████▉                                   | 520/1000 [8:13:38<48:24:12, 363.03s/it]

## Heuristic - N1

In [8]:
model_heur = model.first_solution_heuristic(nodes_limit=1)

In [9]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # set params
    model_heur.set_param_val({"b":b.cpu().numpy()})
    # solve
    tick = time.time()
    try:
        xval, objval = model_heur.solve("scip")
        # eval
        params.append(list(b))
        sols.append(list(list(xval.values())[0].values()))
        objvals.append(objval)
        conviols.append(sum(model_heur.cal_violation()))
    except:
        params.append(list(b))
        sols.append(None)
        objvals.append(None)
        conviols.append(None)
    tock = time.time()
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
print("Number of None values: ", df["Sol"].isna().sum())
df.to_csv("result/nc_heur_n1_20-20.csv")

  3%|██                                                                            | 26/1000 [01:03<1:05:18,  4.02s/it]

ERROR: Solver (scip) returned non-zero return code (4294967295)
ERROR: Solver log: WARNING: unknown parameter <quiet> SCIP version 8.1.0
[precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex
6.0.4] [GitHash: 6129793871] Copyright (c) 2002-2023 Zuse Institute Berlin
(ZIB)

    External libraries:
      Soplex 6.0.4         Linear Programming Solver developed at Zuse
      Institute Berlin (soplex.zib.de) [GitHash: 950b1658] CppAD 20180000.0
      Algorithmic Differentiation of C++ algorithms developed by B. Bell
      (github.com/coin-or/CppAD) ZLIB 1.2.13          General purpose
      compression library by J. Gailly and M. Adler (zlib.net) AMPL/MP
      4e2d45c4     AMPL .nl file reader library (github.com/ampl/mp) PaPILO
      2.1.4         parallel presolve for integer and linear optimization
      (github.com/scipopt/papilo) [GitHash: ee0677c4] bliss 0.77
      Computing Graph Automorphism Groups by T. Junttila and P. Kaski
      (https://users.aalto.fi/~tjuntti

 13%|██████████                                                                     | 128/1000 [04:04<45:48,  3.15s/it]

ERROR: Solver (scip) returned non-zero return code (4294967295)
ERROR: Solver log: WARNING: unknown parameter <quiet> SCIP version 8.1.0
[precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex
6.0.4] [GitHash: 6129793871] Copyright (c) 2002-2023 Zuse Institute Berlin
(ZIB)

    External libraries:
      Soplex 6.0.4         Linear Programming Solver developed at Zuse
      Institute Berlin (soplex.zib.de) [GitHash: 950b1658] CppAD 20180000.0
      Algorithmic Differentiation of C++ algorithms developed by B. Bell
      (github.com/coin-or/CppAD) ZLIB 1.2.13          General purpose
      compression library by J. Gailly and M. Adler (zlib.net) AMPL/MP
      4e2d45c4     AMPL .nl file reader library (github.com/ampl/mp) PaPILO
      2.1.4         parallel presolve for integer and linear optimization
      (github.com/scipopt/papilo) [GitHash: ee0677c4] bliss 0.77
      Computing Graph Automorphism Groups by T. Junttila and P. Kaski
      (https://users.aalto.fi/~tjuntti

 18%|█████████████▉                                                                 | 176/1000 [04:12<01:55,  7.14it/s]

...

ERROR: Solver (scip) returned non-zero return code (4294967295)
ERROR: Solver log: WARNING: unknown parameter <quiet> SCIP version 8.1.0
[precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex
6.0.4] [GitHash: 6129793871] Copyright (c) 2002-2023 Zuse Institute Berlin
(ZIB)

    External libraries:
      Soplex 6.0.4         Linear Programming Solver developed at Zuse
      Institute Berlin (soplex.zib.de) [GitHash: 950b1658] CppAD 20180000.0
      Algorithmic Differentiation of C++ algorithms developed by B. Bell
      (github.com/coin-or/CppAD) ZLIB 1.2.13          General purpose
      compression library by J. Gailly and M. Adler (zlib.net) AMPL/MP
      4e2d45c4     AMPL .nl file reader library (github.com/ampl/mp) PaPILO
      2.1.4         parallel presolve for integer and linear optimization
      (github.com/scipopt/papilo) [GitHash: ee0677c4] bliss 0.77
      Computing Graph Automorphism Groups by T. Junttila and P. Kaski
      (https://users.aalto.fi/~tjuntti

 99%|█████████████████████████████████████████████████████████████████████████████▉ | 986/1000 [18:00<00:02,  4.80it/s]

ERROR: Solver (scip) returned non-zero return code (4294967295)
ERROR: Solver log: WARNING: unknown parameter <quiet> SCIP version 8.1.0
[precision: 8 byte] [memory: block] [mode: optimized] [LP solver: Soplex
6.0.4] [GitHash: 6129793871] Copyright (c) 2002-2023 Zuse Institute Berlin
(ZIB)

    External libraries:
      Soplex 6.0.4         Linear Programming Solver developed at Zuse
      Institute Berlin (soplex.zib.de) [GitHash: 950b1658] CppAD 20180000.0
      Algorithmic Differentiation of C++ algorithms developed by B. Bell
      (github.com/coin-or/CppAD) ZLIB 1.2.13          General purpose
      compression library by J. Gailly and M. Adler (zlib.net) AMPL/MP
      4e2d45c4     AMPL .nl file reader library (github.com/ampl/mp) PaPILO
      2.1.4         parallel presolve for integer and linear optimization
      (github.com/scipopt/papilo) [GitHash: ee0677c4] bliss 0.77
      Computing Graph Automorphism Groups by T. Junttila and P. Kaski
      (https://users.aalto.fi/~tjuntti

100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [18:02<00:00,  1.08s/it]


            Obj Val  Constraints Viol  Elapsed Time
count  9.800000e+02        980.000000   1000.000000
mean   4.156708e+03          0.034875      1.081715
std    9.383635e+04          0.337414      7.112253
min   -3.232752e-01          0.000000      0.075088
25%    1.951013e-01          0.000000      0.096011
50%    6.638513e-01          0.000000      0.155230
75%    5.273073e+00          0.000000      0.186484
max    2.771145e+06          4.584774     60.969856
Number of infeasible solution: 13
Number of None values:  20


## Learnable Rounding

In [7]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [8]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 64            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [9]:
# set problem
import neuromancer as nm
from src.problem import nmNonconvex
from src.func.layer import netFC
from src.func import roundGumbelModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundGumbelModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmNonconvex(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [10]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 595.90
Epoch 1, Validation Loss: 1.45
Epoch 2, Validation Loss: 1.02
Epoch 3, Validation Loss: 1.14
Epoch 4, Validation Loss: 1.05
Epoch 5, Validation Loss: 0.86
Epoch 6, Validation Loss: 0.93
Epoch 7, Validation Loss: 0.75
Epoch 8, Validation Loss: 0.92
Epoch 9, Validation Loss: 0.82
Epoch 10, Validation Loss: 0.92
Epoch 11, Validation Loss: 0.94
Epoch 12, Validation Loss: 0.74
Epoch 13, Validation Loss: 0.74
Epoch 14, Validation Loss: 0.80
Epoch 15, Validation Loss: 0.77
Epoch 16, Validation Loss: 0.71
Epoch 17, Validation Loss: 0.70
Epoch 18, Validation Loss: 0.69
Epoch 19, Validation Loss: 0.67
Epoch 20, Validation Loss: 0.71
Epoch 21, Validation Loss: 0.57
Epoch 22, Validation Loss: 0.65
Epoch 23, Validation Loss: 0.66
Epoch 24, Validation Loss: 0.68
Epoch 25, Validation Loss: 0.67
Epoch 26, Validation Loss: 0.63
Epoch 27, Validation Loss: 0.55
Epoch 28, Validation Loss: 0.63
Epoch 29, Validation Loss: 0.67
Epoch 30, Validation Loss: 0.56
Epoch 31, Valida

In [11]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_lr_20-20.csv")

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 110.10it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean      0.385840          0.000175      0.004287
std       0.120253          0.004202      0.001869
min       0.122742          0.000000      0.001252
25%       0.301175          0.000000      0.003109
50%       0.380554          0.000000      0.004002
75%       0.461874          0.000000      0.004667
max       0.802535          0.123495      0.015047
Number of infeasible solution: 3


## Learnable Threshold

In [12]:
# random seed
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# hyperparameters
penalty_weight = 100  # weight of constraint violation penealty
hlayers_sol = 5       # number of hidden layers for solution mapping
hlayers_rnd = 4       # number of hidden layers for solution mapping
hsize = 64            # width of hidden layers for solution mapping
lr = 1e-3             # learning rate

In [14]:
# set problem
import neuromancer as nm
from src.problem import nmNonconvex
from src.func.layer import netFC
from src.func import roundThresholdModel
# build neural architecture for the solution map
func = nm.modules.blocks.MLP(insize=num_ineq, outsize=num_var, bias=True,
                             linear_map=nm.slim.maps["linear"],
                             nonlin=nn.ReLU, hsizes=[hsize]*hlayers_sol)
smap = nm.system.Node(func, ["b"], ["x"], name="smap")
# define rounding model
layers_rnd = netFC(input_dim=num_ineq+num_var, hidden_dims=[hsize]*hlayers_rnd, output_dim=num_var)
rnd = roundThresholdModel(layers=layers_rnd, param_keys=["b"], var_keys=["x"],  output_keys=["x_rnd"], 
                       int_ind=model.int_ind, continuous_update=True, name="round")
# build neuromancer problem for rounding
components = nn.ModuleList([smap, rnd]).to("cuda")
loss_fn = nmNonconvex(["b", "x_rnd"], num_var, num_ineq, penalty_weight)

In [15]:
from src.problem.neuromancer.trainer import trainer
# training
epochs = 200                    # number of training epochs
warmup = 20                     # number of epochs to wait before enacting early stopping policy
patience = 20                   # number of epochs with no improvement in eval metric to allow before early stopping
optimizer = torch.optim.AdamW(components.parameters(), lr=lr)
# create a trainer for the problem
my_trainer = trainer(components, loss_fn, optimizer, epochs, patience, warmup, device="cuda")
# training for the rounding problem
my_trainer.train(loader_train, loader_dev)

Epoch 0, Validation Loss: 492.62
Epoch 1, Validation Loss: 1.12
Epoch 2, Validation Loss: 0.29
Epoch 3, Validation Loss: 0.26
Epoch 4, Validation Loss: 0.19
Epoch 5, Validation Loss: 0.31
Epoch 6, Validation Loss: 0.21
Epoch 7, Validation Loss: 0.21
Epoch 8, Validation Loss: 0.12
Epoch 9, Validation Loss: 0.17
Epoch 10, Validation Loss: 0.18
Epoch 11, Validation Loss: 0.23
Epoch 12, Validation Loss: 0.20
Epoch 13, Validation Loss: 0.11
Epoch 14, Validation Loss: 0.14
Epoch 15, Validation Loss: 0.08
Epoch 16, Validation Loss: 0.12
Epoch 17, Validation Loss: 0.17
Epoch 18, Validation Loss: 0.13
Epoch 19, Validation Loss: 0.08
Epoch 20, Validation Loss: 0.07
Epoch 21, Validation Loss: 0.08
Epoch 22, Validation Loss: 0.08
Epoch 23, Validation Loss: 0.07
Epoch 24, Validation Loss: 0.09
Epoch 25, Validation Loss: 0.30
Epoch 26, Validation Loss: 0.06
Epoch 27, Validation Loss: 0.08
Epoch 28, Validation Loss: 0.06
Epoch 29, Validation Loss: 0.09
Epoch 30, Validation Loss: 0.05
Epoch 31, Valida

In [16]:
params, sols, objvals, conviols, elapseds = [], [], [], [], []
for b in tqdm(data_test.datadict["b"]):
    # data point as tensor
    datapoints = {"b": torch.unsqueeze(b, 0).to("cuda"), 
                  "name": "test"}
    # infer
    components.eval()
    tick = time.time()
    with torch.no_grad():
        for comp in components:
            datapoints.update(comp(datapoints))
    tock = time.time()
    # assign params
    model.set_param_val({"b":b.cpu().numpy()})
    # assign vars
    x = datapoints["x_rnd"]
    for i in range(num_var):
        model.vars["x"][i].value = x[0,i].item()
    # get solutions
    xval, objval = model.get_val()    
    params.append(list(b.cpu().numpy()))
    sols.append(list(list(xval.values())[0].values()))
    objvals.append(objval)
    conviols.append(sum(model.cal_violation()))
    elapseds.append(tock - tick)
df = pd.DataFrame({"Param":params, "Sol":sols, "Obj Val": objvals, "Constraints Viol": conviols, "Elapsed Time": elapseds})
time.sleep(1)
print(df.describe())
print("Number of infeasible solution: {}".format(np.sum(df["Constraints Viol"] > 0)))
df.to_csv("result/nc_lt_20-20.csv")

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 109.82it/s]


           Obj Val  Constraints Viol  Elapsed Time
count  1000.000000       1000.000000   1000.000000
mean     -0.004955          0.000231      0.004263
std       0.055064          0.004786      0.001784
min      -0.083935          0.000000      0.001504
25%      -0.041584          0.000000      0.003065
50%      -0.021474          0.000000      0.004003
75%       0.014630          0.000000      0.004731
max       0.281345          0.132805      0.014439
Number of infeasible solution: 4
